# agents

> Reinforcement learning agent algorithms

In [ ]:
# | default_exp agents

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
# | hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
# | export

from abc import ABC
from abc import abstractmethod
from collections import deque
import random
from typing import *
from typing import Tuple

from fastcore.basics import patch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# | export


class ReplayBuffer:
    """A simple replay buffer."""

    def __init__(self, max_len: int):
        """Initialize the replay buffer."""

        self.max_len = max_len
        self.memory: deque = deque(maxlen=self.max_len)

    def __len__(self):
        """Return the length of the replay buffer."""

        return len(self.memory)

    def push(self, transition: Tuple[np.ndarray, ...]):
        """Add a transition to the replay buffer."""

        self.memory.append(transition)

    def sample(self, batch_size: int):
        """Sample a batch of transitions from the replay buffer."""

        return random.sample(self.memory, batch_size)

In [ ]:
# | export


class BaseAgent(ABC):
    @abstractmethod
    def get_action(self, state: np.ndarray) -> np.ndarray:
        ...

    @abstractmethod
    def step(self, transition: Tuple[np.ndarray, ...]) -> float:
        ...

    @abstractmethod
    def save(self, path) -> None:
        ...

    @abstractmethod
    def load(self, path) -> None:
        ...

In [ ]:
# | export


class RandomAgent(BaseAgent):
    """A simple random agent."""

    def __init__(self, action_size: Union[int, Tuple[float, float]]):
        """Initialize the agent."""

        self.action_size = action_size

    def get_action(self, state: np.ndarray):
        """Get an action from the agent."""

        if isinstance(self.action_size, int):
            return np.random.randint(self.action_size)
        if isinstance(self.action_size, tuple):
            return np.random.uniform(self.action_size[0], self.action_size[1])

    def step(self, transition: Tuple[np.ndarray, ...]):
        """Take a step in the environment."""

        return np.NaN

In [ ]:
# | export


class QNet(nn.Module):
    """A simple deep Q-network."""

    def __init__(self, input_size: int, output_size: int, hidden_layers: List[int]):
        """Initialize the Q-network."""

        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        self.layers.extend(
            [
                nn.Linear(hidden_layers[i - 1], hidden_layers[i])
                for i in range(1, len(hidden_layers))
            ]
        )
        self.layers.append(nn.Linear(hidden_layers[-1], output_size))

    def forward(self, x):
        """Forward pass through the Q-network."""

        for i in range(len(self.layers)):
            x = self.layers[i](x)
            x = torch.relu(x)

        return x

In [ ]:
# | export


class DDQNAgent(BaseAgent):
    """A simple deep double Q-learning agent."""

    def __init__(
        self,
        state_size: int,
        action_size: int,
        hidden_layers: List[int],
        lr: float,
        epsilon: float,
        epsilon_min: float,
        epsilon_decay: float,
        gamma: float,
        update_freq: int,
        target_copy_freq: int,
        train_loops: int,
        buffer_max: int,
        buffer_min: int,
        batch_size: int,
    ):
        """Initialize the agent."""

        self.state_size = state_size
        self.action_size = action_size
        self.hidden_layers = hidden_layers
        self.lr = lr
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.gamma = gamma
        self.update_freq = update_freq
        self.target_copy_freq = target_copy_freq
        self.train_loops = train_loops
        self.buffer_max = buffer_max
        self.buffer_min = buffer_min
        self.batch_size = batch_size

        self.buffer = ReplayBuffer(max_len=self.buffer_max)

        self.q = QNet(self.state_size, self.action_size, self.hidden_layers)
        self.q_target = QNet(self.state_size, self.action_size, self.hidden_layers)
        self.optimizer = optim.Adam(self.q.parameters(), lr=self.lr)
        self.step_count = 0

    def _update_target(self):
        """Update the target network."""

        self.q_target.load_state_dict(self.q.state_dict())

    def _update_epsilon(self):
        """Update the epsilon value."""

        self.epsilon = max(
            self.epsilon_min, self.epsilon * (1 - self.epsilon_decay) * self.step_count
        )

    def _update(self, transitions):
        """Update the Q-network."""

        state, action, reward, next_state, done_mask = transitions
        state = torch.from_numpy(state).float()
        action = torch.from_numpy(action).float()
        reward = torch.from_numpy(reward).float()
        next_state = torch.from_numpy(next_state).float()
        done_mask = torch.from_numpy(done_mask).float()

        q_out = self.q(state)
        q_action = q_out.gather(1, action)
        max_q_prime = self.q_target(next_state).max(1)[0].unsqueeze(1)
        target = reward + self.gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_action, target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()

    def get_action(self, state):
        """Get an action from the agent."""

        state = torch.from_numpy(state)
        out = self.q(state).max(dim=1)

        if random.random() < self.epsilon:
            return random.randint(0, self.action_size - 1)
        else:
            return out.argmax().item()

    def step(self, env):
        """Take a step in the environment."""

        action = self.get_action(env.state)

        state, action, reward, next_state, done, info = env.step(action)
        done_mask = 1 if done else 0
        transition = (
            state,
            np.array(action),
            np.array(reward),
            next_state,
            np.array(done_mask),
        )

        self.buffer.push(transition)

        if len(self.buffer) >= self.buffer_min:
            for _ in range(self.train_loops):
                transitions = self.buffer.sample(self.batch_size)
                self._update(transitions)

        if self.step_count % self.target_copy_freq == 0:
            self._update_target()

        self._update_epsilon()
        self.step_count += 1

        return done

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()